In [1]:
from tkinter import Tk, Canvas, Frame, Button, Label, BOTH, TOP, BOTTOM, StringVar
from tkinter import *
from tkinter import OptionMenu
import time
import re

MARGIN = 20  # Pixels around the board
SIDE = 50  # Width of every board cell.
WIDTH = HEIGHT = MARGIN * 2 + SIDE * 9  # Width and height of the whole board


In [2]:


# Create your Sudoku solving functions
def backtracking_csp(sudoku_game):
    # Implement Backtracking CSP solving algorithm
    if solve_backtracking(sudoku_game.puzzle):
        print("Sudoku puzzle solved successfully using Backtracking algorithm:")
        sudoku_game.game_over = True
        return True
    else:
        print("No solution exists for the Sudoku puzzle using Backtracking algorithm.")
        return False

def solve_backtracking(board):
    empty_cell = find_empty_cell(board)
    if not empty_cell:
        return True  # All cells are filled, puzzle solved
        
    row, col = empty_cell
    for num in range(1, 10):
        if is_valid_move(board, row, col, num):
            board[row][col] = num  # Try placing num in the empty cell
            #time.sleep(2)  # Introduce delay
            if solve_backtracking(board):
                return True  # If puzzle solved, return True
            
            board[row][col] = 0  # Backtrack if puzzle not solved
    
    return False  # No solution found

def find_empty_cell(board):
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                return (row, col)
    return None

def is_valid_move(board, row, col, num):
    # Check if num is not present in current row, column, and 3x3 box
    return (
        not used_in_row(board, row, num) and 
        not used_in_col(board, col, num) and 
        not used_in_box(board, row - row % 3, col - col % 3, num)
    )

def used_in_row(board, row, num):
    return num in board[row]

def used_in_col(board, col, num):
    return any(row[col] == num for row in board)

def used_in_box(board, box_start_row, box_start_col, num):
    for row in range(3):
        for col in range(3):
            if board[row + box_start_row][col + box_start_col] == num:
                return True
    return False

def arc_consistency_csp(sudoku_game):
    # Implement Arc-consistency CSP solving algorithm
    if solve_arc_consistency(sudoku_game.puzzle):
        print("Sudoku puzzle solved successfully using Arc-consistency algorithm:")
        sudoku_game.game_over = True
        return True
    else:
        print("No solution exists for the Sudoku puzzle using Arc-consistency algorithm.")
        return False

def solve_arc_consistency(board):
    # Initialize queue with all arcs (each arc represents a constraint between two variables)
    arcs = []
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                for num in range(1, 10):
                    arcs.append(((row, col), num))

    while arcs:
        cell, num = arcs.pop(0)
        row, col = cell

        if is_valid_move(board, row, col, num):
            board[row][col] = num  # Assign the value
            #time.sleep(2)  # Introduce delay
            # Enforce arc-consistency by propagating constraints
            if not enforce_arc_consistency(board, row, col, num):
                return False  # Inconsistent solution, backtrack
            # Add new arcs resulting from the assigned value
            new_arcs = []
            for r in range(9):
                for c in range(9):
                    if board[r][c] == 0 and (r != row or c != col):
                        new_arcs.append(((r, c), num))
            arcs.extend(new_arcs)

    return True

def enforce_arc_consistency(board, row, col, num):
    # Check row, column, and 3x3 subgrid for inconsistency
    if not all_constraints_satisfied(board, row, col):
        return False

    # Propagate constraints to neighboring cells in the same row, column, and 3x3 subgrid
    if not propagate_constraints(board, row, col, num):
        return False

    return True

def all_constraints_satisfied(board, row, col):
    # Check row
    if board[row].count(board[row][col]) > 1:
        return False

    # Check column
    if [board[r][col] for r in range(9)].count(board[row][col]) > 1:
        return False

    # Check 3x3 subgrid
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    if any(
        [
            board[r][c] == board[row][col]
            for r in range(start_row, start_row + 3)
            for c in range(start_col, start_col + 3)
            if (r, c) != (row, col)
        ]
    ):
        return False

    return True

def propagate_constraints(board, row, col, num):
    # Propagate constraints to neighboring cells in the same row
    for c in range(9):
        if c != col and board[row][c] == 0:
            if not revise(board, row, c, num):
                return False

    # Propagate constraints to neighboring cells in the same column
    for r in range(9):
        if r != row and board[r][col] == 0:
            if not revise(board, r, col, num):
                return False

    # Propagate constraints to neighboring cells in the same 3x3 subgrid
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for r in range(start_row, start_row + 3):
        for c in range(start_col, start_col + 3):
            if (r, c) != (row, col) and board[r][c] == 0:
                if not revise(board, r, c, num):
                    return False

    return True

def revise(board, row, col, num):
    # Remove num from domain of (row, col) if no valid assignment exists
    if num in get_domain(board, row, col):
        board[row][col] = 0
        if not any(
            [
                is_valid_move(board, row, col, n)
                for n in get_domain(board, row, col)
                if n != num
            ]
        ):
            return False
    return True

def get_domain(board, row, col):
    domain = set(range(1, 10))
    for r in range(9):
        domain.discard(board[r][col])  # Remove values in the same column
    for c in range(9):
        domain.discard(board[row][c])  # Remove values in the same row
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for r in range(start_row, start_row + 3):
        for c in range(start_col, start_col + 3):
            domain.discard(board[r][c])  # Remove values in the same 3x3 subgrid
    return domain

class SudokuError(Exception):
    """
    An application specific error.
    """
    pass


class SudokuUI(Frame):
    """
    The Tkinter UI, responsible for drawing the board and accepting user input.
    """
    def __init__(self, parent, game):
        self.game = game
        Frame.__init__(self, parent)
        self.parent = parent

        self.row, self.col = -1, -1
        self.difficulty_label = StringVar()  # Variable to hold the selected difficulty level
        self.selected_difficulty = StringVar(value="Easy")  # Variable to hold the selected difficulty
        self.selected_puzzle = StringVar(value="Puzzle 1")  # Variable to hold the selected puzzle option
        self.selected_algorithm = StringVar(value="Backtracking")

        self.__initUI()

    def __initUI(self):
        self.parent.title("Sudoku")
        self.pack(fill=BOTH)
        self.canvas = Canvas(self, width=WIDTH, height=HEIGHT)
        self.canvas.pack(fill=BOTH, side=TOP)

        button_frame = Frame(self)
        button_frame.pack(fill=BOTH, side=BOTTOM, padx=240, pady=0)

        # Solve button
        solve_button = Button(button_frame, text="Solve", command=self.__solve)
        solve_button.pack(side=LEFT)

        # Reset button
        reset_button = Button(button_frame, text="Reset", command=self.__clear_answers)
        reset_button.pack(side=LEFT, padx=50)
        
        # Add label to display selected difficulty
        difficulty_frame = Frame(self)
        difficulty_frame.pack(fill=BOTH, side=BOTTOM, padx=10)

        Label(difficulty_frame, text="Select Level:  ").pack(side=LEFT)

        # Options for difficulty levels
        difficulty_options = ["Easy", "Medium", "Hard"]
        # Create a dropdown menu for selecting difficulty
        OptionMenu(difficulty_frame, self.selected_difficulty, *difficulty_options, command=self.__set_difficulty).pack(side=LEFT)
        self.selected_difficulty.set(difficulty_options[0])  # Set default difficulty

        # Label to display selected difficulty
        self.difficulty_label_label = Label(difficulty_frame, textvariable=self.difficulty_label)
        self.difficulty_label_label.pack(side=LEFT)

        # Add label to select algorithm
        algorithm_frame = Frame(self)
        algorithm_frame.pack(fill=BOTH, side=BOTTOM, padx=10)

        Label(algorithm_frame, text="Select Algorithm:").pack(side=LEFT)

        # Radio buttons for selecting algorithm
        Radiobutton(algorithm_frame, text="Backtracking", variable=self.selected_algorithm, value="Backtracking", command=self.__set_algorithm).pack(side=LEFT)
        Radiobutton(algorithm_frame, text="Arc-consistency", variable=self.selected_algorithm, value="Arc-consistency", command=self.__set_algorithm).pack(side=LEFT)

        # Add label to choose puzzles
        puzzle_frame = Frame(self)
        puzzle_frame.pack(fill=BOTH, side=BOTTOM, padx=10)
        Label(puzzle_frame, text="Choose Puzzle:").pack(side=LEFT)

        # Options for puzzle levels
        puzzle_options = ["Puzzle 1", "Puzzle 2", "Puzzle 3", "Puzzle 4"]
        self.puzzle_buttons = {}
        for option in puzzle_options:
            button = Radiobutton(puzzle_frame, text=option, variable=self.selected_puzzle, value=option, command=self.__set_puzzle)
            button.pack(side=LEFT)
            self.puzzle_buttons[option] = button

        self.__draw_grid()
        self.__draw_puzzle()

        self.canvas.bind("<Button-1>", self.__cell_clicked)
        self.canvas.bind("<Key>", self.__key_pressed)

    def __set_puzzle(self):
        selected_puzzle = self.selected_puzzle.get()
        puzzle_number = int(selected_puzzle.split()[-1])  # Extract the puzzle number from the selected option
        difficulty = self.selected_difficulty.get().lower()  # Get the selected difficulty
        self.game.set_puzzle(puzzle_number, difficulty)  # Call the set_puzzle method with puzzle number and difficulty
        print("Level:",difficulty)
        print("Puzzle :",puzzle_number)
        if(difficulty=="easy" and puzzle_number==1):
            puzzle_file = game.board_files["easy"][0]  # Select the first puzzle file by default
            with open(puzzle_file, 'r') as boards_file:
        # Correctly initialize SudokuBoard with a file path string
            #game.start_puzzle = SudokuBoard(puzzle_file).board
                game.start_puzzle = SudokuBoard(puzzle_file).board
                game.game_over = False
                game.puzzle = [row[:] for row in game.start_puzzle]
                self.reset_board()
        
                
        elif(difficulty=="easy" and puzzle_number==2):
            puzzle_file = game.board_files["easy"][1]  # Select the first puzzle file by default
            with open(puzzle_file, 'r') as boards_file:
        # Correctly initialize SudokuBoard with a file path string
                game.start_puzzle = SudokuBoard(puzzle_file).board
                game.game_over = False
                game.puzzle = [row[:] for row in game.start_puzzle]
                self.reset_board()
        elif(difficulty=="easy" and puzzle_number==3):
            puzzle_file = game.board_files["easy"][2]  # Select the first puzzle file by default
            with open(puzzle_file, 'r') as boards_file:
        # Correctly initialize SudokuBoard with a file path string
                game.start_puzzle = SudokuBoard(puzzle_file).board
                game.game_over = False
                game.puzzle = [row[:] for row in game.start_puzzle]
                self.reset_board()
        elif(difficulty=="easy" and puzzle_number==4):
            puzzle_file = game.board_files["easy"][3]  # Select the first puzzle file by default
            with open(puzzle_file, 'r') as boards_file:
        # Correctly initialize SudokuBoard with a file path string
                game.start_puzzle = SudokuBoard(puzzle_file).board
                game.game_over = False
                game.puzzle = [row[:] for row in game.start_puzzle]
                self.reset_board()      
        elif(difficulty=="medium" and puzzle_number==1):
            puzzle_file = game.board_files["medium"][0]  # Select the first puzzle file by default
            with open(puzzle_file, 'r') as boards_file:
        # Correctly initialize SudokuBoard with a file path string
                game.start_puzzle = SudokuBoard(puzzle_file).board
                game.game_over = False
                game.puzzle = [row[:] for row in game.start_puzzle]
                self.reset_board()  
        elif(difficulty=="medium" and puzzle_number==2):
            puzzle_file = game.board_files["medium"][1]  # Select the first puzzle file by default
            with open(puzzle_file, 'r') as boards_file:
        # Correctly initialize SudokuBoard with a file path string
                game.start_puzzle = SudokuBoard(puzzle_file).board
                game.game_over = False
                game.puzzle = [row[:] for row in game.start_puzzle]
                self.reset_board()
        elif(difficulty=="medium" and puzzle_number==3):
            puzzle_file = game.board_files["medium"][2]  # Select the first puzzle file by default
            with open(puzzle_file, 'r') as boards_file:
        # Correctly initialize SudokuBoard with a file path string
                game.start_puzzle = SudokuBoard(puzzle_file).board
                game.game_over = False
                game.puzzle = [row[:] for row in game.start_puzzle]
                self.reset_board()
        elif(difficulty=="medium" and puzzle_number==4):
            puzzle_file = game.board_files["medium"][3]  # Select the first puzzle file by default
            with open(puzzle_file, 'r') as boards_file:
        # Correctly initialize SudokuBoard with a file path string
                game.start_puzzle = SudokuBoard(puzzle_file).board
                game.game_over = False
                game.puzzle = [row[:] for row in game.start_puzzle]
                self.reset_board()
        elif(difficulty=="hard" and puzzle_number==1):
            puzzle_file = game.board_files["hard"][0]  # Select the first puzzle file by default
            with open(puzzle_file, 'r') as boards_file:
        # Correctly initialize SudokuBoard with a file path string
                game.start_puzzle = SudokuBoard(puzzle_file).board
                game.game_over = False
                game.puzzle = [row[:] for row in game.start_puzzle]
                self.reset_board()
        elif(difficulty=="hard" and puzzle_number==2):
            puzzle_file = game.board_files["hard"][1]  # Select the first puzzle file by default
            with open(puzzle_file, 'r') as boards_file:
        # Correctly initialize SudokuBoard with a file path string
                game.start_puzzle = SudokuBoard(puzzle_file).board
                game.game_over = False
                game.puzzle = [row[:] for row in game.start_puzzle]
                self.reset_board()
        elif(difficulty=="hard" and puzzle_number==3):
            puzzle_file = game.board_files["hard"][2]  # Select the first puzzle file by default
            with open(puzzle_file, 'r') as boards_file:
        # Correctly initialize SudokuBoard with a file path string
                game.start_puzzle = SudokuBoard(puzzle_file).board
                game.game_over = False
                game.puzzle = [row[:] for row in game.start_puzzle]
                self.reset_board()
        elif(difficulty=="hard" and puzzle_number==4):
            puzzle_file = game.board_files["hard"][3]  # Select the first puzzle file by default
            with open(puzzle_file, 'r') as boards_file:
        # Correctly initialize SudokuBoard with a file path string
                game.start_puzzle = SudokuBoard(puzzle_file).board
                game.game_over = False
                game.puzzle = [row[:] for row in game.start_puzzle]
                self.reset_board()
        

    # The rest of the SudokuUI class remains the same


        # Function to update the difficulty level
    def __set_difficulty(self, *args):
        selected_difficulty = self.selected_difficulty.get().lower()
        self.game.set_difficulty(selected_difficulty)
        self.difficulty_label.set(f"Difficulty: {selected_difficulty.capitalize()}")
        self.__set_puzzle()


    def reset_board(self):
    # Clear the Sudoku board canvas
        self.canvas.delete("numbers")

    # Draw the Sudoku board grid
        for i in range(10):
            color = "blue" if i % 3 == 0 else "gray"

            x0 = MARGIN + i * SIDE
            y0 = MARGIN
            x1 = MARGIN + i * SIDE
            y1 = HEIGHT - MARGIN
            self.canvas.create_line(x0, y0, x1, y1, fill=color)

            x0 = MARGIN
            y0 = MARGIN + i * SIDE
            x1 = WIDTH - MARGIN
            y1 = MARGIN + i * SIDE
            self.canvas.create_line(x0, y0, x1, y1, fill=color)

    # Fill in the cells with puzzle data
        for i in range(9):
            for j in range(9):
                answer = self.game.puzzle[i][j]
                if answer!= 0:
                    x = MARGIN + j * SIDE + SIDE / 2
                    y = MARGIN + i * SIDE + SIDE / 2
                    original = self.game.start_puzzle[i][j]
                    color = "black" if answer == original else "sea green"
                    self.canvas.create_text(
                        x, y, text=answer, tags="numbers", fill=color
                    )


    def __set_algorithm(self):
        selected_algorithm = self.selected_algorithm.get()
        print("Selected algorithm:", selected_algorithm)

    def __solve(self):
        selected_algorithm = self.selected_algorithm.get()
        x=self.selected_difficulty.get()
        y=self.selected_puzzle.get()
        # Measure start time
        start_time = time.time()
        if selected_algorithm == "Backtracking":
            print(x)
            print(y)
            backtracking_csp(self.game)
        elif selected_algorithm == "Arc-consistency":
            arc_consistency_csp(self.game)
        # Measure end time
        end_time = time.time()
        
        # Calculate time taken
        solving_time = end_time - start_time

        # Update UI with solving time
        # Update UI with solving time
        solving_time_label = Label(
        self.parent, text=f"Time taken: {solving_time:.2f} seconds", font=("Helvetica", 12, "bold"),
        borderwidth=2, relief="solid"
        )
        solving_time_label.pack(side=BOTTOM)

        # After solving, update the UI to reflect the solved puzzle
        self.__draw_puzzle()
        if self.game.check_win():
            self.__draw_victory()

    def __draw_grid(self):
        """
        Draws grid divided with blue lines into 3x3 squares
        """
        for i in range(10):
            color = "blue" if i % 3 == 0 else "gray"

            x0 = MARGIN + i * SIDE
            y0 = MARGIN
            x1 = MARGIN + i * SIDE
            y1 = HEIGHT - MARGIN
            self.canvas.create_line(x0, y0, x1, y1, fill=color)

            x0 = MARGIN
            y0 = MARGIN + i * SIDE
            x1 = WIDTH - MARGIN
            y1 = MARGIN + i * SIDE
            self.canvas.create_line(x0, y0, x1, y1, fill=color)

    
    def __draw_puzzle(self):
        self.canvas.delete("numbers")
        for i in range(9):
            for j in range(9):
                answer = self.game.puzzle[i][j]
                if answer!= 0:
                    x = MARGIN + j * SIDE + SIDE / 2
                    y = MARGIN + i * SIDE + SIDE / 2
                    original = self.game.start_puzzle[i][j]
                    color = "black" if answer == original else "red"
                    self.canvas.create_text(
                        x, y, text=answer, tags="numbers", fill=color
                    )

    def __draw_cursor(self):
        self.canvas.delete("cursor")
        if self.row >= 0 and self.col >= 0:
            x0 = MARGIN + self.col * SIDE + 1
            y0 = MARGIN + self.row * SIDE + 1
            x1 = MARGIN + (self.col + 1) * SIDE - 1
            y1 = MARGIN + (self.row + 1) * SIDE - 1
            self.canvas.create_rectangle(
                x0, y0, x1, y1,
                outline="red", tags="cursor"
            )

    def __draw_victory(self):
        x0 = y0 = MARGIN + SIDE * 2
        x1 = y1 = MARGIN + SIDE * 7   # Add 40 pixels padding to the right
        self.canvas.create_oval(
            x0, y0, x1, y1,
            tags="victory", fill="dark orange", outline="orange"
        )
        x = y = MARGIN + 4 * SIDE + SIDE / 2
        self.canvas.create_text(
            x, y,
            text="You win!", tags="victory",
            fill="white", font=("Arial", 32),
            #padx=0  # Add 40 pixels padding to the right
        )


    def __cell_clicked(self, event):
        if self.game.game_over:
            return
        x, y = event.x, event.y
        if (MARGIN < x < WIDTH - MARGIN and MARGIN < y < HEIGHT - MARGIN):
            self.canvas.focus_set()

            row, col = int((y - MARGIN) / SIDE), int((x - MARGIN) / SIDE)

            if (row, col) == (self.row, self.col):
                self.row, self.col = -1, -1
            elif self.game.puzzle[row][col] == 0:
                self.row, self.col = row, col
        else:
            self.row, self.col = -1, -1

        self.__draw_cursor()

    def __key_pressed(self, event):
        if self.game.game_over:
            return
        if self.row >= 0 and self.col >= 0 and event.char in "1234567890":
            self.game.puzzle[self.row][self.col] = int(event.char)
            self.col, self.row = -1, -1
            self.__draw_puzzle()
            self.__draw_cursor()
            if self.game.check_win():
                self.__draw_victory()

    def __clear_answers(self):
        self.game.start()
        self.canvas.delete("victory")
        self.__draw_puzzle()
        # Hide the time label
        self.hide_time_label()
    def hide_time_label(self):
        for widget in self.parent.winfo_children():
            if isinstance(widget, Label) and "Time taken:" in widget.cget("text"):
                widget.pack_forget()
class SudokuBoard(object):
    def __init__(self, board_file):
        self.board = self.__create_board(board_file)
        assert isinstance(board_file, str), "Expected file path as a string"
        with open(board_file, 'r') as file:
            # Process the file contents here
            pass

    def __create_board(self, board_file):
        with open(board_file, 'r') as file:
            lines = file.readlines()

        sudoku_board = []
        for line in lines:
            line = line.strip()
            if len(line) < 9:
                line = line.ljust(9, '.')  # Pad short lines with dots
            elif len(line) > 9:
                line = line[:9]  # Trim excess characters
            # Replace empty cells with 0
            line = [int(char) if char.isdigit() else 0 for char in line]
            sudoku_board.append(line)

        return sudoku_board



class SudokuGame(object):
    def __init__(self, difficulty):
        self.board_files = {
            "easy": ["easy-puzzle-1.sudoku", "easy-puzzle-2.sudoku", "easy-puzzle-3.sudoku", "easy-puzzle-4.sudoku"],
            "medium": ["medium-puzzle-1.sudoku", "medium-puzzle-2.sudoku", "medium-puzzle-3.sudoku", "medium-puzzle-4.sudoku"],
            "hard": ["hard-puzzle-1.sudoku", "hard-puzzle-2.sudoku", "hard-puzzle-3.sudoku", "hard-puzzle-4.sudoku"],
        }
        self.start_puzzle = []
        self.game_over = False
        self.puzzle = []

        self.set_difficulty(difficulty)

    def set_difficulty(self, level):
        self.board_files[level]
        puzzle_file = self.board_files[level][0]  # Select the first puzzle file by default
        self.start_puzzle = SudokuBoard(puzzle_file).board
        self.puzzle = [row[:] for row in self.start_puzzle]
        self.game_over = False

    def set_puzzle(self, puzzle_number, difficulty):
        if puzzle_number >= 0 and puzzle_number <= 3:
            puzzle_file = self.board_files[difficulty][puzzle_number - 1]  # Subtract 1 to account for 0-based indexing
            self.start_puzzle = SudokuBoard(puzzle_file).board
            self.puzzle = [row[:] for row in self.start_puzzle]
            self.game_over = False



    def start(self):
        self.game_over = False
        self.puzzle = [row[:] for row in self.start_puzzle]  # Reset puzzle to the start state

    # Remaining methods...

    def check_win(self):
        for row in range(9):
            if not self.__check_row(row):
                return False
        for column in range(9):
            if not self.__check_column(column):
                return False
        for row in range(3):
            for column in range(3):
                if not self.__check_square(row, column):
                    return False
        self.game_over = True
        return True

    def __check_block(self, block):
        return set(block) == set(range(1, 10))

    def __check_row(self, row):
        return self.__check_block(self.puzzle[row])

    def __check_column(self, column):
        return self.__check_block(
            [self.puzzle[row][column] for row in range(9)]
        )

    def __check_square(self, row, column):
        return self.__check_block(
            [
                self.puzzle[r][c]
                for r in range(row * 3, (row + 1) * 3)
                for c in range(column * 3, (column + 1) * 3)
            ]
        )


def validate_sudoku_puzzle_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    for line in lines:
        if len(line.strip())!= 9:
            print(f"Invalid line in {file_path}: {line}")
            return False
    
    return True

import os


print(os.getcwd())
# Correctly pass the difficulty level as a string
game = SudokuGame("easy")  # Change "easy" to "medium" or "hard" as needed
  # or "medium" or "hard"

# Load the puzzle file based on the difficulty level
# Load the puzzle file based on the difficulty level
puzzle_file = game.board_files["easy"][0]  # Select the first puzzle file by default
with open(puzzle_file, 'r') as boards_file:
    # Correctly initialize SudokuBoard with a file path string
    game.start_puzzle = SudokuBoard(puzzle_file).board
    game.game_over = False
    game.puzzle = [row[:] for row in game.start_puzzle]


    root = Tk()
    ui=SudokuUI(root, game)
    root.geometry("%dx%d" % (WIDTH, HEIGHT + 40))
    # Set default values for selected puzzle and algorithm
    ui.selected_difficulty.set("Easy")
    ui.selected_puzzle.set("Puzzle 1")
    ui.selected_algorithm.set("Backtracking")
    root.mainloop()


c:\Users\Classic\Desktop\AI_Project\AI_Semester_Project\Sudoko_Puzzle_Solver
